In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Iris 데이터셋 로드
iris = load_iris()
X = iris.data
y = iris.target

# One-hot 인코딩
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# 데이터셋을 학습용과 테스트용으로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1017) 

In [2]:
import numpy as np
import random

def sigmoid(z):
        return 1 / (1 + np.exp(-z))
    
def sigmoid_deriv(z):
        return sigmoid(z) * (1 - sigmoid(z))
    
def softmax(z):
    exp_z = np.exp(z)
    return exp_z / np.sum(exp_z)

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate): #p,q,r
        self.input_size = input_size #p
        self.hidden_size = hidden_size #q
        self.output_size = output_size #r
        self.learning_rate = learning_rate
        
        # weight initialize & shape construction
        np.random.seed(202)
        self.W1 = np.random.randn(self.input_size, self.hidden_size) #p*q
        np.random.seed(107)
        self.b1 = np.zeros((self.hidden_size)) #q*1
        np.random.seed(1138)
        self.W2 = np.random.randn(self.hidden_size, self.output_size) #q*r
        np.random.seed(27)
        self.b2 = np.zeros((self.output_size)) #r*1
    
    def forward(self, X): #forward propagation
        self.Z1 = X @ self.W1 + self.b1 #q*1
        self.A1 = sigmoid(self.Z1) #q*1
        self.Z2 = self.A1 @ self.W2 + self.b2 #r*1
        self.A2 = softmax(self.Z2) #r*1
        return self.A2

########################################################################
    def backward(self, X, y, y_pred): #backpropagation
        L_Z2 = (y - y_pred)*(-1)
        A1_Z1 = sigmoid_deriv(self.A1) #q*1
        
        grad_W2 = np.zeros((self.hidden_size,self.output_size)) #q*r
        for q in range(self.hidden_size):
            for r in range(self.output_size):
                grad_W2[q,r]=L_Z2[r]*self.A1[q]
        grad_b2 = np.zeros(self.output_size) #r*1
        for r in range(self.output_size):
            grad_b2[r]=L_Z2[r]*1

        grad_W1 = np.zeros((self.input_size,self.hidden_size)) #p*q
        for p in range(self.input_size):
            for q in range(self.hidden_size):
                for r in range(self.output_size):
                    grad_W1[p,q]+=L_Z2[r]*self.W2[q,r]*A1_Z1[q]*X[p]
        grad_b1 = np.zeros(self.hidden_size) #q*1
        for p in range(self.input_size):
            for q in range(self.hidden_size):
                for r in range(self.output_size):
                    grad_b1[q]+=L_Z2[r]*self.W2[q,r]*A1_Z1[q]*1        

        # update weight & bias
        self.W2 -= grad_W2 * self.learning_rate
        self.b2 -= grad_b2 * self.learning_rate
        self.W1 -= grad_W1 * self.learning_rate
        self.b1 -= grad_b1 * self.learning_rate
        
########################################################################
        
    def Train(self, X, y, epochs):
        loss_set=list()
        for n in range(len(X)):
            y_pred = self.forward(X[n,:])
            loss = -np.sum(y[n,:]*np.log(y_pred))
            loss_set.append(loss)
        avgloss = np.mean(loss_set)
        print(f'Epoch {0}, Loss: {avgloss}')        
        for epoch in range(epochs):
            
            loss_set=list()
            for n in range(len(X)):
                y_pred = self.forward(X[n,:])
                loss = -np.sum(y[n,:]*np.log(y_pred)) #Cross Entropy Loss
                self.backward(X[n,:], y[n,:], y_pred) 
                loss_set.append(loss)
            avgloss = np.mean(loss_set)
            
            if (epoch+1) % 5 == 0: print(f'Epoch {epoch+1}, Loss: {avgloss}')
           
    def Test(self, X):
        testoutput=[]
        for n in range(len(X)):
            y_pred = self.forward(X[n,:])
            testoutput.append(np.argmax(y_pred))    
        return testoutput

In [3]:
#setting hyperparameters
learning_rate=0.005
epochs=30

NN=NeuralNetwork(4,7,3,learning_rate)
NN.Train(X_train,y_train,epochs)

Epoch 0, Loss: 1.1799459307487936
Epoch 5, Loss: 0.7697343605798745
Epoch 10, Loss: 0.6724774620576618
Epoch 15, Loss: 0.5865647960528668
Epoch 20, Loss: 0.5445126310389149
Epoch 25, Loss: 0.5078813746853241
Epoch 30, Loss: 0.46463978978466247


In [4]:
testoutput = NN.Test(X_test)
y_test_labels = np.argmax(y_test, axis=1)

#accuracy
accuracy = np.mean(testoutput == y_test_labels)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 96.67%


In [5]:
for i in range(10):
    pred=NN.forward(X_test[i,:])
    target=y_test[i,:]
    print(pred,target,np.argmax(pred)==np.argmax(target))

[0.79087513 0.17280196 0.03632291] [1. 0. 0.] True
[0.0147225  0.34725775 0.63801976] [0. 0. 1.] True
[0.01470303 0.33480121 0.65049575] [0. 0. 1.] True
[0.83338793 0.13828392 0.02832815] [1. 0. 0.] True
[0.00902271 0.33238056 0.65859673] [0. 0. 1.] True
[0.17068413 0.52608564 0.30323023] [0. 1. 0.] True
[0.8299676  0.14205715 0.02797525] [1. 0. 0.] True
[0.01058652 0.32585278 0.6635607 ] [0. 0. 1.] True
[0.81611671 0.15257764 0.03130565] [1. 0. 0.] True
[0.13539729 0.5567226  0.30788011] [0. 1. 0.] True
